In [1]:
from preprocess import *
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, LSTM, Activation
from keras.utils import to_categorical
import wandb
from wandb.keras import WandbCallback
import matplotlib.pyplot as plt
import sklearn.metrics as metrics

Using TensorFlow backend.


In [2]:
wandb.init()
config = wandb.config

config.max_len = 21
config.buckets = 50

# Save data to array file first
save_data_to_array(max_len=config.max_len, n_mfcc=config.buckets)

#labels=np.array(["chirping_birds", "crickets", "crow", 
#                 "frog", "insects"])
labels=np.array(["AAT", "AHV", "AMA", 
                 "ART", "ASI", "AVH",
                "AVT"])

Saving vectors of label - 'AVT': 100%|███████████████████████████████████████████████| 222/222 [00:04<00:00, 50.72it/s]


In [3]:
# Loading train/test set
X_train, X_test, X_val, y_train, y_test, y_val = get_train_test()

In [4]:
# Setting channels to 1 to generalize stereo sound to 1 channel
channels = 1
config.epochs = 50
config.batch_size = 100

# Number of classes
num_classes = 7

# Reshape X_train and X_test to include a 4th dimension (channels)
X_train = X_train.reshape(X_train.shape[0], config.buckets, config.max_len, channels)
X_test = X_test.reshape(X_test.shape[0], config.buckets, config.max_len, channels)
X_val = X_val.reshape(X_val.shape[0], config.buckets, config.max_len, channels)

In [5]:
# Spectrogram visualized of 0th element
print(X_train.shape)
#plt.imshow(X_train[500, :, :, 0])

(169, 50, 21, 1)


In [6]:
# Getting vector number where each number corresponds to a label
y_train_hot = to_categorical(y_train)
y_test_hot = to_categorical(y_test)
y_val_hot = to_categorical(y_val)

In [7]:
# Building the model
model = Sequential()

input_shape= (config.buckets, config.max_len, channels)

model.add(Conv2D(24, (3, 3), strides=(1, 1), input_shape=input_shape))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))
model.add(Activation('relu'))

model.add(Conv2D(48, (3, 3), padding="valid"))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))
model.add(Activation('relu'))

model.add(Conv2D(48, (3, 1), padding="valid"))
model.add(Activation('relu'))

model.add(Flatten())
model.add(Dropout(rate=0.5))

model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(rate=0.5))

model.add(Dense(len(labels)))
model.add(Activation('softmax'))
model.summary()
# Conv2D: 
#    Filters: 32
#    Kernel_size: (3,3) (height/width of the 2D convolution window)     
'''model.add(Conv2D(32, (3, 3),
    input_shape=(config.buckets, config.max_len, channels),
    activation='relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))'''


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 48, 19, 24)        240       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 9, 24)         0         
_________________________________________________________________
activation_1 (Activation)    (None, 24, 9, 24)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 22, 7, 48)         10416     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 11, 3, 48)         0         
_________________________________________________________________
activation_2 (Activation)    (None, 11, 3, 48)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 9, 3, 48)        

"model.add(Conv2D(32, (3, 3),\n    input_shape=(config.buckets, config.max_len, channels),\n    activation='relu'))\n\nmodel.add(MaxPooling2D(pool_size=(2, 2)))\n\nmodel.add(Flatten())\n\nmodel.add(Dense(128, activation='relu'))\nmodel.add(Dense(num_classes, activation='softmax'))"

In [8]:
# Configure CNN for training
model.compile(loss="categorical_crossentropy",
                  optimizer="adam",
                  metrics=['accuracy'])

In [9]:
wandb.init()
print(y_train_hot.shape)
print(labels.shape)
print(X_train.shape)
# Train the CNN model
#    X_train: Input data
#    y_train_hot: Target data
model.fit(X_train, y_train_hot, epochs=config.epochs, validation_data=(X_val, y_val_hot), callbacks=[WandbCallback(data_type="image", labels=labels)])

Failed to connect to W&B servers after 10 seconds.                    Letting user process proceed while attempting to reconnect.


(169, 7)
(7,)
(169, 50, 21, 1)

Train on 169 samples, validate on 113 samples
Epoch 1/50
169/169 [==============================] - ETA: 3s - loss: 7.2267 - accuracy: 0.12 - ETA: 1s - loss: 5.7420 - accuracy: 0.15 - ETA: 0s - loss: 4.6317 - accuracy: 0.25 - ETA: 0s - loss: 3.7351 - accuracy: 0.34 - 1s 8ms/step - loss: 3.6083 - accuracy: 0.3609 - val_loss: 1.2447 - val_accuracy: 0.6991
Epoch 2/50
169/169 [==============================] - ETA: 0s - loss: 1.5250 - accuracy: 0.50 - ETA: 0s - loss: 1.6566 - accuracy: 0.50 - ETA: 0s - loss: 1.7947 - accuracy: 0.47 - ETA: 0s - loss: 1.8819 - accuracy: 0.46 - 0s 2ms/step - loss: 1.9045 - accuracy: 0.4675 - val_loss: 1.2482 - val_accuracy: 0.6991
Epoch 3/50
169/169 [==============================] - ETA: 0s - loss: 1.7073 - accuracy: 0.43 - ETA: 0s - loss: 2.0419 - accuracy: 0.39 - ETA: 0s - loss: 1.8637 - accuracy: 0.43 - ETA: 0s - loss: 1.8119 - accuracy: 0.40 - ETA: 0s - loss: 1.7048 - accuracy: 0.43 - 0s 3ms/step - loss: 1.6837 - accuracy:

169/169 [==============================] - ETA: 0s - loss: 0.4177 - accuracy: 0.81 - ETA: 0s - loss: 0.5882 - accuracy: 0.72 - ETA: 0s - loss: 0.6115 - accuracy: 0.74 - ETA: 0s - loss: 0.5959 - accuracy: 0.76 - 0s 2ms/step - loss: 0.6150 - accuracy: 0.7574 - val_loss: 0.6825 - val_accuracy: 0.8053
Epoch 28/50
169/169 [==============================] - ETA: 0s - loss: 0.6774 - accuracy: 0.78 - ETA: 0s - loss: 0.6044 - accuracy: 0.80 - 0s 814us/step - loss: 0.5800 - accuracy: 0.8107 - val_loss: 0.6592 - val_accuracy: 0.8142
Epoch 29/50
169/169 [==============================] - ETA: 0s - loss: 0.4753 - accuracy: 0.84 - ETA: 0s - loss: 0.4826 - accuracy: 0.84 - ETA: 0s - loss: 0.4482 - accuracy: 0.86 - ETA: 0s - loss: 0.5395 - accuracy: 0.82 - 0s 2ms/step - loss: 0.5262 - accuracy: 0.8284 - val_loss: 0.7025 - val_accuracy: 0.8053
Epoch 30/50
169/169 [==============================] - ETA: 0s - loss: 0.2894 - accuracy: 0.90 - ETA: 0s - loss: 0.3292 - accuracy: 0.89 - ETA: 0s - loss: 0.4548

In [10]:
# Save the keras model
model.save("ant_cnn_model.h5")
print("Model has been saved.")

Model has been saved.


## Running the IntelliChirp Biophony CNN

In [11]:
from keras.models import load_model

# Load the model
loaded_model = load_model('ant_cnn_model.h5')

In [12]:
# Summarize the model
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 48, 19, 24)        240       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 9, 24)         0         
_________________________________________________________________
activation_1 (Activation)    (None, 24, 9, 24)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 22, 7, 48)         10416     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 11, 3, 48)         0         
_________________________________________________________________
activation_2 (Activation)    (None, 11, 3, 48)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 9, 3, 48)         

In [16]:
y_pred_ohe = loaded_model.predict(X_test)  # shape=(n_samples, 12)
y_pred_labels = np.argmax(y_pred_ohe, axis=1)  # only necessary if output has one-hot-encoding, shape=(n_samples)

confusion_matrix = metrics.confusion_matrix(y_true=y_test, y_pred=y_pred_labels)  # shape
print(confusion_matrix)

for class_i in range(len(labels)) :
    indices = np.argwhere(y_test == class_i)
    sum = 0
    for index in indices:
        sum += (y_test[index] == y_pred_labels[index])
    if(len(indices) > 0) : mean = sum/len(indices)
    else : mean = "N/A"
    print("Accuracy for class", labels[class_i], ":", mean)

print("Overall Accuracy :", np.mean(y_test == y_pred_labels))

[[ 7  0  0  0  0  8]
 [ 0  1  0  0  0  2]
 [ 0  0  1  0  0  7]
 [ 0  0  0  0  0  1]
 [ 0  0  0  0  3  2]
 [ 2  0  1  0  0 36]]
Accuracy for class AAT : [0.46666667]
Accuracy for class AHV : [0.33333333]
Accuracy for class AMA : [0.125]
Accuracy for class ART : 0
Accuracy for class ASI : [0.]
Accuracy for class AVH : [0.6]
Accuracy for class AVT : [0.92307692]
Overall Accuracy : 0.676056338028169


'Accuracy for class chirping_birds : [0.8]\nAccuracy for class crickets : [0.25]\nAccuracy for class crow : [0.83333333]\nAccuracy for class frog : [0.3]\nAccuracy for class insects : [0.85714286]\nOverall Accuracy : 0.525'

In [17]:
## Running the model

n_mfcc = config.buckets
max_len = config.max_len
# convert file to wav2mfcc
# Mel-frequency cepstral coefficients
file_path = "./prediction/nature_sc.wav"
big_wave, sr = librosa.load(file_path, mono=True, sr=None)
#print(wave.shape, sr)

classification = []

for sec_index in range( int(big_wave.shape[0] / sr) ) :
    start_sec = sec_index
    end_sec = sec_index + 1
    
    sec_to_trim = np.array( [ float(start_sec), float(end_sec) ] )
    print(sec_to_trim)
    sec_to_trim = np.ceil( sec_to_trim * sr )

    wave = big_wave[int(sec_to_trim[0]) : int(sec_to_trim[1])]
    print(wave)

    wave = np.asfortranarray(wave[::3])
    mfcc = librosa.feature.mfcc(wave, sr=16000, n_mfcc=n_mfcc)

    # If maximum length exceeds mfcc lengths then pad the remaining ones
    if (max_len > mfcc.shape[1]):
        pad_width = max_len - mfcc.shape[1]
        mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')

    # Else cutoff the remaining parts
    else:
        mfcc = mfcc[:, :max_len]

    # Convert wav to MFCC
    prediction_data = wav2mfcc('./prediction/nature_sc.wav')
    prediction_data = mfcc
    print(prediction_data.shape)
    #print(wav2mfcc())
    # Reshape to 4 dimensions
    prediction_data = prediction_data.reshape(1, config.buckets, config.max_len, channels)
    #prediction_data = prediction_data.reshape(1, 20, config.max_len, channels)

    # Run the model on the inputted file
    predicted = loaded_model.predict(prediction_data)

    # Output the prediction values for each class
    print ('PREDICTED VALUES')
    labels_indices = range(len(labels))
    max_value = 0
    max_value_index = 0
    for index in labels_indices:
        print('\n', labels[index], ": ", '%.08f' % predicted[0,index])
        if predicted[0,index] > max_value:
            max_value_index = index
            max_value = predicted[0,index]

    # Output the prediction
    if max_value < 0.5:
        print("GUESS: Nothing")
        classification.append( { "class" : "Nothing", "timestamp" : start_sec } )
    else:
        print('\n\nGUESS: ', labels[max_value_index])
        classification.append( { "class" : labels[max_value_index], "timestamp" : start_sec } )

print(classification)

[0. 1.]
[ 0.0000000e+00  1.5258789e-05  0.0000000e+00 ...  3.3020020e-02
  1.2680054e-02 -8.7432861e-03]
(50, 21)
PREDICTED VALUES

 AAT :  0.01039583

 AHV :  0.02998595

 AMA :  0.02692612

 ART :  0.00021275

 ASI :  0.00109535

 AVH :  0.00171419

 AVT :  0.92966986


GUESS:  AVT
[1. 2.]
[-0.03717041 -0.05769348 -0.06455994 ...  0.01766968  0.01895142
  0.01779175]
(50, 21)
PREDICTED VALUES

 AAT :  0.09363972

 AHV :  0.27155691

 AMA :  0.11288872

 ART :  0.00317154

 ASI :  0.00362689

 AVH :  0.00379819

 AVT :  0.51131809


GUESS:  AVT
[2. 3.]
[ 0.02345276  0.02101135  0.01712036 ... -0.01161194 -0.0141449
 -0.01431274]
(50, 21)
PREDICTED VALUES

 AAT :  0.02302537

 AHV :  0.34436873

 AMA :  0.12974481

 ART :  0.00235426

 ASI :  0.00554969

 AVH :  0.01236655

 AVT :  0.48259062
GUESS: Nothing
[3. 4.]
[-0.01583862 -0.01066589 -0.00762939 ... -0.0377655  -0.03556824
 -0.02685547]
(50, 21)
PREDICTED VALUES

 AAT :  0.00185250

 AHV :  0.01309109

 AMA :  0.00708705

 ART : 

 AHV :  0.00321765

 AMA :  0.00698048

 ART :  0.00000044

 ASI :  0.00000350

 AVH :  0.00000469

 AVT :  0.98979086


GUESS:  AVT
[31. 32.]
[-0.04328918 -0.03413391 -0.03421021 ...  0.05908203  0.06370544
  0.05949402]
(50, 21)
PREDICTED VALUES

 AAT :  0.00053715

 AHV :  0.00938936

 AMA :  0.00308272

 ART :  0.00004391

 ASI :  0.00017127

 AVH :  0.00006393

 AVT :  0.98671162


GUESS:  AVT
[32. 33.]
[ 0.06063843  0.06056213  0.06610107 ... -0.12741089 -0.13371277
 -0.12313843]
(50, 21)
PREDICTED VALUES

 AAT :  0.00000053

 AHV :  0.00012500

 AMA :  0.00003068

 ART :  0.00000000

 ASI :  0.00000027

 AVH :  0.00000024

 AVT :  0.99984336


GUESS:  AVT
[33. 34.]
[-0.09968567 -0.06376648 -0.03105164 ... -0.0138092  -0.01574707
 -0.01896667]
(50, 21)
PREDICTED VALUES

 AAT :  0.00000420

 AHV :  0.00023515

 AMA :  0.00047268

 ART :  0.00000002

 ASI :  0.00000135

 AVH :  0.00000821

 AVT :  0.99927837


GUESS:  AVT
[34. 35.]
[-0.00811768  0.00149536  0.00953674 ... -0.004776